Project 2: Classification of Parkinsons Disease Speech Dataset

By Christian Johnston, CPSC 4383 Artificial Intelligence, Fall 2023

In this project, I will be classifying a dataset on speech patterns to help improve diagnosis for parkinsons through telemedicine as a cheap and non-invasive option for patients.

The method of classification that I will be using is a Support Vector Machine (SVM) implemented from sci-kit learn and I will be using a sample parkinsons speech classification dataset.  This method will draw from the shlokKh github codebase as well as the scikit learn documentation database: 
https://github.com/shlokKh/Parkinsons-Voice-Detection/blob/master/parkinsonsdata.py , 
https://scikit-learn.org/stable/modules/svm.html#support-vector-machines , 
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

In [30]:
import pandas as pd
import numpy as np
import os, sys

#assemble the toolbox.
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#acquire and preprocess data.
#make sure to convert the data location string to a raw string with r, otherwise it could cause unicode errors.
data = pd.read_csv(r'C:\Users\Christian\Desktop\School Stuff\Fall 2023\AI\parkinsonsData.csv')
predictors = data.drop(['name'], axis = 1)
predictors = predictors.drop(['status'], axis = 1)
target = data['status']

#here the data is scaled to simplify the data and scale the outliers linearly down to a fixed range, namely -1 to 1.
scaler = MinMaxScaler((-1, 1))
X = scaler.fit_transform(predictors)
Y = target

#I am using the recommended 0.25 or a 1/4 test size split for the data and a random state number for consistency.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 7)

Here I will implement the SVM and make a prediction on the dataset.  The SVC(Support Vector Classification) model I am using fits a kernel function to the dataset and applies a linear separation hyper-plane to the data on a 2 dimensional plane of data on voice deviation in parkinson's diagnoses compared to a control group.  We want the linear separation to have a maximum distance margin between the two classifications.  In this kernel function we use the default SVC, which uses a linear kernel and default C value of 1.0 for the weight of the penalty of samples found within margin boundaries.  We can adjust the C value, or regularization parameter, to control the tradeoff between misclassifications and the width of the margin.  There are also many different kernel functions that can be applied to support vector machines for classification depending on the distribution of data and we can test some of these as well using scikit learn's SVC model functionalities later in the project.

In [31]:
model = SVC()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.8979591836734694
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        10
           1       0.89      1.00      0.94        39

    accuracy                           0.90        49
   macro avg       0.94      0.75      0.80        49
weighted avg       0.91      0.90      0.88        49

[[ 5  5]
 [ 0 39]]


As we can see from the test results, the accuracy of this predictive model is about 90%.  Also included is our confusion matrix, and here we can see that we have 5 true positives, 5 false positives, 0 false negatives, and 39 true negatives Which reflect the total number of data points in this test, 49.  In this data table, the 0 index for the first row represents our positive diagnosis, whereas the 1 is the indicator of our negative diagnosis.  Is it then possible to manipulate our test parameters to get higher accuracy?  Lets experiment and try some other values for C and then a different kernel entirely.

In [69]:
model = SVC(C = 0.5)
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.8775510204081632
              precision    recall  f1-score   support

           0       1.00      0.40      0.57        10
           1       0.87      1.00      0.93        39

    accuracy                           0.88        49
   macro avg       0.93      0.70      0.75        49
weighted avg       0.89      0.88      0.86        49

[[ 4  6]
 [ 0 39]]


Here we see the new changed values for our C parameter are from the default of 1 to a lower 0.5 value.  The strength is inversely proportional to the value so smaller values should result in higher reglarization.  Our confusion matrix also changed, with the test moving one of the true positives to a false positive.  In this, the results show that our accuracy went down to ~88% along with the scores of precision, sensitivity, and F1.  Increasing the strength of C shows that we get less accuracy and scores across all metrics have fallen.  The hyperplane margins are stretched larger and will accomodate more misclassifications and we can see that with one of our true positives being classified as a false positive in the confusion matrix.

In [68]:
model = SVC(C = 2)
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.9183673469387755
              precision    recall  f1-score   support

           0       1.00      0.60      0.75        10
           1       0.91      1.00      0.95        39

    accuracy                           0.92        49
   macro avg       0.95      0.80      0.85        49
weighted avg       0.93      0.92      0.91        49

[[ 6  4]
 [ 0 39]]


Here, I increase the values of the C parameter to 2.  This causes the optimization to chose a smaller margin for the hyperplane, thus increasing the classification strength and lower reglarization.  This is reflected by a proportional change in the rate of true positives to false positives.  The accuracy increases to ~92%  and the scores are reflecting higher values in their metrics.  This is because one of the false positives has been reclassified as a true positive due to the shrinking of the hyperplane and shorter values of the orthogonal vector for misdiagnosis.  We have to be careful not to choose too high of a value or we might get overfitting and misclassification can start to occur.

The next experiment I want to try is modifying the kernel to a different algorithm.  Sometimes the data distribution does not allow for a linear classification so we can try other kernels to try and get a better fit.  Lets try a polynomial kernel but we again have to be careful of overfitting with too high of a degree so we will start with 3.  I am also using the default gamma which scales based on 1 / (n_features * X.var()).  All other parameters of the algorithm are the scikit learn default values for polynomial SVC.

In [55]:
model2 = SVC(C = 1, kernel = 'poly', degree = 3)
model2.fit(X_train, Y_train)

y_pred = model2.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.8979591836734694
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        10
           1       0.89      1.00      0.94        39

    accuracy                           0.90        49
   macro avg       0.94      0.75      0.80        49
weighted avg       0.91      0.90      0.88        49

[[ 5  5]
 [ 0 39]]


The results show similar values to our first model, so perhaps a polynomial kernel is not needed in this case as the linear kernel seems to handle the data distribution just fine.  Lets do another test and try a higher degree polynomial to try and fit the data better.

In [70]:
model2 = SVC(C = 1, kernel = 'poly', degree = 5)
model2.fit(X_train, Y_train)

y_pred = model2.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.9591836734693877
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.95      1.00      0.97        39

    accuracy                           0.96        49
   macro avg       0.98      0.90      0.93        49
weighted avg       0.96      0.96      0.96        49

[[ 8  2]
 [ 0 39]]


In this, we can see that the accuracy goes way up when using a higher polynomial for the degree of the kernel.  Our model now gives 8 true positives and only 2 false positives while retaining our perfect precision on negatives.  This in turn gives us an accuracy of about 96% which is much better than our linear fit.  Can we go higher without causing any errors in classification?

In [73]:
model2 = SVC(C = 1, kernel = 'poly', degree = 7)
model2.fit(X_train, Y_train)

y_pred = model2.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.9591836734693877
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.97      0.97      0.97        39

    accuracy                           0.96        49
   macro avg       0.94      0.94      0.94        49
weighted avg       0.96      0.96      0.96        49

[[ 9  1]
 [ 1 38]]


Now we can see that we have introduced a false negative value, even though our false positive values have fallen.  This drops our precision values and shows we are starting to overfit.  Is there a good medium then?

In [74]:
model2 = SVC(C = 1, kernel = 'poly', degree = 6)
model2.fit(X_train, Y_train)

y_pred = model2.predict(X_test)

print("Support Vector Machine: ")
print(metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))
print(metrics.confusion_matrix(Y_test, y_pred))

Support Vector Machine: 
0.9795918367346939
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.97      1.00      0.99        39

    accuracy                           0.98        49
   macro avg       0.99      0.95      0.97        49
weighted avg       0.98      0.98      0.98        49

[[ 9  1]
 [ 0 39]]


We now have much better precision, and our accuracy goes up to 98%.  It seems like a polynomial of degree 6 is our best fit for our data.  We reduced the false positives to only 1 in 10 and improved our metrics across the board.  We can now use this to classify our data accurately, at least with this specific subset as specified in the train-test split.

In conclusion I hope this project showcases some of the benefits of using SVM for a real application of data in a dataset, especially when used with a polynomial kernel function to classify the data.  As a side note, I spent a large amount of time attempting to visualize the data with pyplot but I had a difficult time getting the visualization to work and for that I apologize, I did not manage my time well enough for that application.  This dataset is a little tricky to visualize as the visualization tutorials I saw used very different data sets and variables that I was not able to properly follow or apply to this data.  When I attempted to construct a scatter plot, I encountered an issue I could not resolve and therefore could not visualize the data with plots.  In the future I will spend more time working on visualization to help display the data in a more intuitive way and I hope the tables of metrics is enough to show the results of the testing.